In [30]:
import numpy as np
import random
import matplotlib.pyplot as plt

In [31]:
from environment.fogg_behavioural_model import Patient 

Basic enviroment where the patients emotional state and sleep pattern remains unchanged by the prompts.
Here we are intenting just to figure out when prompt is effective and learn optimal difficulty of activity. 

In [32]:
testPatient = Patient(20, True)

In [33]:
testPatient.get_ability(1,1)

1

If patient performed activity once dayily and otherwise was not incorrectly notified the total reward in a day would be:

    - 10 for correct behaviour
    - and 23 * 0.1 for correct restrain from notification

In [34]:
DAyilyGoodReward = 10 + 23 * 0.1 

In [35]:
DAyilyGoodReward

12.3

In [36]:
DAyilyGoodReward * 31

381.3

# Random

In [37]:
hours = 24 * 31
total_reward = 0
for episode in range(1, hours+1):
#     state = testPatient.reset()
    
    prompt = random.choice([0,1])
    activity_difficulty = random.choice([0,1])
    activity_length = random.choice([0,1])
    action = (prompt, activity_difficulty, activity_length )
    state, reward= testPatient.step(action)
    total_reward+=reward

print('Hours:{} Reward after a month:{}'.format(episode, total_reward))

Hours:744 Reward after a month:-209.40000000000106


# Model Free Approach

# Q-learner

In [56]:
testPatient2 = Patient(20, True)

In [57]:
testPatient2.__dict__.keys()

dict_keys(['behaviour_threshold', 'has_family', 'week_days', 'hours', 'activity_suggested', 'activity_performed', 'time_of_the_day', 'day_of_the_week', 'motion_activity_list', 'awake_list', 'last_activity_score', 'location', 'valence', 'arousal', 'cognitive_load'])

In [58]:
promptN = 2 # yes/no
difficultyN = 2 # easy /dificult
lengthN = 2 # short/ long
actionN = 6

time_of_the_day = 24
day_of_the_week = 7
activity_score = 2# low/ high
location = 2 # home/ other
sleeping = 2 # yes/no
valence = 2 # positive/negative
arousal = 2 # low/high
motion = 3 #stationary, walking, driving
cognitive_load = 2 # low/ high

q_table = np.zeros([time_of_the_day, day_of_the_week,activity_score,location, sleeping,valence,arousal,motion,cognitive_load, actionN ])

In [59]:
def class2action(a):
    if a == 0:
        return 0,0,0
    elif a==1:
        return 0,0,1
    elif a ==2:
        return 0,1,1
    elif a == 3:
        return 1,1,1
    elif a == 4:
        return 1,1,0
    else:
        return 1,0,1

In [60]:
def action2class(a):
    if a == (0,0,0):
        return 0
    elif a == (0,0,1):
        return 1
    elif a == (0,1,1):
        return 2
    elif a == (1,1,1):
        return 3
    elif a == (1,1,0):
        return 4
    else:
        return 5

In [61]:
len(testPatient2._get_current_state())

9

In [62]:

# Hyperparameters
alpha = 0.1
gamma = 0.5
epsilon = 0.5

# For plotting metrics
all_epochs = []
total_rewardQL = []
state = testPatient2._get_current_state()
days = 31
for _ in range(1, days):
#     testPatient2.reset()
#     state = testPatient2._get_current_state()
    
    for h in range(1, 24):

 
        if random.uniform(0, 1) < epsilon:
                # Explore action space
            prompt = random.choice([0,1])
            activity_difficulty = random.choice([0,1])
            activity_length = random.choice([0,1])
            action = (prompt, activity_difficulty, activity_length )
        
            next_state, reward = testPatient2.step(action) 
            action = action2class(action)
        else:
            action  = np.argmax(q_table[state[0],state[1], state[2],state[3], state[4],state[5], state[6],state[7],state[8]]) # Exploit learned values
            a = class2action(action)
            next_state, reward = testPatient2.step(a) 
        
        
        if reward ==10:
            print(action)
            print(state)
        old_value = q_table[state[0],state[1], state[2],state[3], state[4],state[5], state[6],state[7],state[8], action]
        next_max = np.max(q_table[next_state[0], next_state[1],next_state[2], next_state[3],next_state[4], next_state[5],
                                  next_state[6], next_state[7], next_state[8]])

        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state[0],state[1], state[2],state[3], state[4],state[5], state[6],state[7],state[8], action] = new_value

        total_rewardQL.append(reward)
        state = next_state
   

    

5
(5, 6, 1, 1, 0, 1, 0, 1, 0)
5
(9, 6, 1, 1, 0, 1, 1, 1, 0)
4
(11, 0, 1, 1, 0, 1, 1, 1, 0)
5
(6, 5, 1, 1, 0, 1, 0, 1, 0)
5
(10, 1, 1, 1, 0, 1, 1, 1, 0)
4
(10, 3, 1, 1, 0, 1, 1, 1, 0)
5
(11, 4, 0, 1, 0, 1, 1, 1, 0)


In [64]:
np.sum(total_rewardQL)

-24.5

In [65]:
 #time_of_the_day, day_of_the_week,activity_score,location, sleeping,valence,arousal,motion,cognitive_load, actionN
np.argmax(q_table[13,6, 0,0, 0,0, 0,0,0])

0